### Imports

In [36]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from tqdm import tqdm
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from TorchCRF import CRF

%run updatePreprocessing.ipynb

[['ق', 'و', 'ل', 'ه'], ['ل', 'ع', 'د', 'م'], ['م', 'ا'], ['ت', 'ت', 'ع', 'ل', 'ق'], ['إ', 'ل', 'خ'], ['أ', 'ي'], ['ا', 'ل', 'و', 'ص', 'ي', 'ة'], ['ق', 'و', 'ل', 'ه'], ['م', 'ا'], ['م', 'ر'], ['أ', 'ي'], ['ق', 'ب', 'ي', 'ل'], ['ق', 'و', 'ل'], ['ا', 'ل', 'م', 'ت', 'ن'], ['ل', 'غ', 'ت'], ['و', 'ل', 'و'], ['ا', 'ق', 'ت', 'ص', 'ر'], ['ع', 'ل', 'ى'], ['أ', 'و', 'ص', 'ي', 'ت'], ['ل', 'ه'], ['ب', 'ش', 'ا', 'ة'], ['أ', 'و'], ['أ', 'ع', 'ط', 'و', 'ه'], ['ش', 'ا', 'ة'], ['و', 'ل', 'ا'], ['غ', 'ن', 'م'], ['ل', 'ه'], ['ع', 'ن', 'د'], ['ا', 'ل', 'م', 'و', 'ت'], ['ه', 'ل'], ['ت', 'ب', 'ط', 'ل'], ['ا', 'ل', 'و', 'ص', 'ي', 'ة'], ['أ', 'و'], ['ي', 'ش', 'ت', 'ر', 'ى'], ['ل', 'ه'], ['ش', 'ا', 'ة'], ['و', 'ي', 'ؤ', 'خ', 'ذ'], ['م', 'ن'], ['ق', 'و', 'ل', 'ه'], ['ا', 'ل', 'آ', 'ت', 'ي'], ['ك', 'م', 'ا'], ['ل', 'و'], ['ل', 'م'], ['ي', 'ق', 'ل'], ['م', 'ن'], ['م', 'ا', 'ل', 'ي'], ['و', 'ل', 'ا'], ['م', 'ن'], ['غ', 'ن', 'م', 'ي'], ['أ', 'ن', 'ه', 'ا'], ['ل', 'ا'], ['ت', 'ب', 'ط', 'ل'], ['و', 'ع', 'ب', 'ا', 'ر',

### Constants

In [52]:
EMBEDDING_DIM = 200
HIDDEN_SIZE = 512
NUM_LAYERS = 1
NUM_EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 256
VOCAB_SIZE = len(basic_arabic_letters) + 1
LABELS_SIZE = len(DIACRITICS)

TRAIN_PATH = "../dataset/train.txt"
VAL_PATH = "../dataset/val.txt"
LSTM_PATH="./models/lstm.pth"
RNN_PATH="./models/rnn.pth"
CNN_PATH = "./models/cnn.pth"
CRF_Val_PATH="./models/crf_val.pth"
CRF_PATH="./models/crf.pth"
CNN_val_PATH="./models/cnn_val.pth"

### Model building

### RNN

In [38]:
class RNN(nn.Module):
    def __init__(self, vocab_size, n_classes, embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        """
        The constructor of our RNN model
        Inputs:
        - vacab_size: the number of unique characters
        - embedding_dim: the embedding dimension
        - n_classes: the number of final classes (diacritics)
        """
        super(RNN, self).__init__()

        # (1) Create an embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # (2) Create an LSTM layer with hidden size = hidden_size and batch_first = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        
        # (3) Create a linear layer with number of neorons = n_classes
        self.linear = nn.Linear(hidden_size * 2, n_classes)

    def forward(self, sentences):
        """
        This function does the forward pass of our model
        Inputs:
        - sentences: tensor of shape (batch_size, max_length)

        Returns:
        - final_output: tensor of shape (batch_size, max_length, n_classes)
        """

        final_output = None
        
        embeddings = self.embedding(sentences)
        lstm_out, _ = self.lstm(embeddings)
        output = self.linear(lstm_out)
        # final_output = F.softmax(output, dim=1)
        return output

### CNN

In [39]:
class CNN(nn.Module):
    def __init__(self, vocab_size, n_classes, embedding_dim=EMBEDDING_DIM):
        super(CNN, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Convolutional layers
        self.conv1 = nn.Conv1d(embedding_dim, 256, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1)

        # Max pooling layers
        # self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)

        # Fully connected layers
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, n_classes)

    def forward(self, x):
        print("1", x.shape)
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        print("2", x.shape)
        # Convolutional layers with ReLU activation and max pooling
        x = F.relu(self.conv1(x))
        # x = self.pool(x)
        print("3", x.shape)
        x = F.relu(self.conv2(x))
        # x = self.pool(x)
        print("4", x.shape)
        x = F.relu(self.conv3(x))
        # x = self.pool(x)
        print("5", x.shape)

        # Fully connected layers with ReLU activation
        x = x.view(-1, 128)
        x = F.relu(self.fc1(x))
        print("6", x.shape)
        x = self.fc2(x)
        print("7", x.shape)

        return x

In [48]:
class RNN_CNN(nn.Module):
    def __init__(self, vocab_size, n_classes, embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        super(RNN_CNN, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # CNN layer
        self.conv1d = nn.Conv1d(in_channels=embedding_dim, out_channels=64, kernel_size=3, padding=1)  # Adjust parameters as needed

        # LSTM layer
        self.lstm = nn.LSTM(64, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        # Linear layer
        self.linear = nn.Linear(hidden_size * 2, n_classes)

    def forward(self, sentences):
        embeddings = self.embedding(sentences)

        # Pass embeddings through CNN layer
        conv_out = self.conv1d(embeddings.permute(0, 2, 1))  # Reshape for CNN
        conv_out = torch.relu(conv_out)
        conv_out = conv_out.permute(0, 2, 1)  # Reshape back for LSTM

        # Pass CNN output through LSTM layer
        lstm_out, _ = self.lstm(conv_out)

        # Final output layer
        output = self.linear(lstm_out)
        return output


### CRF

In [40]:
class LSTM_CRF(nn.Module):
    def __init__(self, vocab_size, n_classes, embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        super(LSTM_CRF, self).__init__()
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # BiLSTM layer
        self.bilstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        # CRF layer
        self.crf = CRF(n_classes)

        # Linear layer
        self.linear = nn.Linear(hidden_size * 2, n_classes)
        # Create a CRF layer
        self.crf = CRF(n_classes, batch_first=True)

    def forward(self, sentences):
        # Input x is a sequence of indices
        embedded = self.embedding(sentences)

        # BiLSTM layer
        lstm_out, _ = self.bilstm(embedded)

        # Linear layer for classification
        linear_out = self.linear(lstm_out)
        if labels is not None:
            # Calculate the negative log-likelihood loss using the CRF layer
            loss = self.crf(output, labels)
            return -loss  # Return negative log-likelihood as we usually minimize it during training
        else:
            # If labels are not provided, return the raw output
            return output


In [41]:
class RNN_CRF(nn.Module):
    def __init__(self, vocab_size, n_classes, embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        super(RNN_CRF, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        # Linear layer
        self.linear = nn.Linear(hidden_size * 2, n_classes)

        # CRF layer
        self.crf = CRF(n_classes)  # Place the CRF layer after the linear layer

    def forward(self, sentences):
        embeddings = self.embedding(sentences)
        lstm_out, _ = self.lstm(embeddings)
        output = self.linear(lstm_out)
        return output  # Return raw output for CRF loss calculation

    def predict(self, sentences):
        output = self.forward(sentences)
        predictions = self.crf.decode(output)
        return predictions


### Train

In [42]:

def train(model, path,val_path, train_dataset, train_labels, val_dataset,val_labels,batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, learning_rate=LEARNING_RATE):
    """
    This function implements the training logic
    Inputs:
    - model: the model to be trained
    - train_dataset: the training set
    - batch_size: integer represents the number of examples per step
    - epochs: integer represents the total number of epochs (full training pass)
    - learning_rate: the learning rate to be used by the optimizer
    """

    # (1) create the dataloader of the training set (make the shuffle=True)
    tensor_train_dataset = TensorDataset(train_dataset, train_labels)
    train_dataloader = DataLoader(tensor_train_dataset, batch_size=batch_size, shuffle=True)
    tensor_val_dataset = TensorDataset(val_dataset, val_labels)
    val_dataloader = DataLoader(tensor_val_dataset, batch_size=batch_size, shuffle=False)

    # (2) make the criterion cross entropy loss
    criterion = torch.nn.CrossEntropyLoss()

    # (3) create the optimizer (Adam)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # (4) create a learning rate scheduler (optional but recommended)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Adjust parameters as needed

    # GPU configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
    
    best_accuracy = 0.0
    best_accuracy_val=0.0
    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        
        for train_input, train_label in tqdm(train_dataloader):
            # Zero your gradients
            optimizer.zero_grad()

            # Move the train input to the device
            train_label = train_label.to(device)

            # Move the train label to the device
            train_input = train_input.to(device)

            # Do the forward pass
            output = model(train_input).float()

            # Loss calculation
            batch_loss = criterion(output.view(-1, output.shape[-1]), train_label.view(-1))

            # Append the batch loss to the total_loss_train
            total_loss_train += batch_loss.item()
            
            # Calculate the batch accuracy (just add the number of correct predictions)
            # Compare predicted diacritic with true diacritic and count correct predictions
            correct_predictions = (output.argmax(dim=2) == train_label)

            # Calculate accuracy for the current batch
            acc = correct_predictions.sum().item()
            total_acc_train += acc

            # Do the backward pass
            batch_loss.backward()

            # Update the weights with your optimizer
            optimizer.step()     
        # Step the learning rate scheduler
        scheduler.step()
        # Calculate the epoch loss
        epoch_loss = total_loss_train / len(train_dataset)

        # Calculate the accuracy
        epoch_acc = total_acc_train / (len(train_dataset) * len(train_dataset[0]))

        print(f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
            | Train Accuracy: {epoch_acc}\n')
        
        if epoch_acc > best_accuracy:
            best_accuracy = epoch_acc
            torch.save(model.state_dict(), path)
            print(f'Saved the best model with accuracy: {best_accuracy} to {path}\n')
         # Validation
        model.eval()  # Set the model to evaluation mode
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():
            for val_input, val_label in tqdm(val_dataloader):
                val_label = val_label.to(device)
                val_input = val_input.to(device)

                output = model(val_input)
                batch_loss = criterion(output.view(-1, output.shape[-1]), val_label.view(-1))
                total_loss_val += batch_loss.item()

                correct_predictions = (output.argmax(dim=2) == val_label)
                acc = correct_predictions.sum().item()
                total_acc_val += acc

        epoch_loss_val = total_loss_val / len(val_dataloader)
        epoch_acc_val = total_acc_val / (len(val_dataset) * len(val_dataset[0]))

        print(f'Epochs: {epoch_num + 1} | Validation Loss: {epoch_loss_val} | Validation Accuracy: {epoch_acc_val}')

        if epoch_acc_val > best_accuracy_val:
            best_accuracy_val = epoch_acc_val
            torch.save(model.state_dict(), val_path)
            print(f'Saved the best model with validation accuracy: {best_accuracy_val} to {val_path}')
        model.train()


In [43]:
corpus=  readFile(TRAIN_PATH)

x_train = []
y_train = []

for sentence in corpus[:1000]:
	# Clean each sentence in the corpus
	# Get the char list for each word in the sentence and its corresponding diacritics
	char_list, diacritics_list = separate_words_and_diacritics(sentence.strip())

	x_train.append(char_list)
	y_train.append(diacritics_list)

X_train_padded = [torch.tensor([char_to_index[char] for char in word]) for sentence in x_train for word in sentence]
X_train_padded = pad_sequence(X_train_padded, batch_first=True)

y_train_padded = [torch.tensor([diacritic_to_index[char] for char in word]) for sentence in y_train for word in sentence]
y_train_padded = pad_sequence(y_train_padded, batch_first=True)

In [44]:
valid_corpus = readFile(VAL_PATH)

X_val = []
y_val = []

for sentence in valid_corpus[:50]:
	# Clean each sentence in the corpus
	# Get the char list for each word in the sentence and its corresponding diacritics
	char_list, diacritics_list = separate_words_and_diacritics(sentence.strip())

	X_val.append(char_list)
	y_val.append(diacritics_list)

X_val_padded = [torch.tensor([char_to_index[char] for char in word]) for sentence in X_val for word in sentence ]
X_val_padded = pad_sequence(X_val_padded, batch_first=True)

y_val_padded = [torch.tensor([diacritic_to_index[char] for char in word]) for sentence in y_val for word in sentence ]
# print(y_val_padded)
y_val_padded = pad_sequence(y_val_padded, batch_first=True)
# print(y_val_padded)

In [49]:
split=y_val_padded.shape[0]//2
y_val=y_val_padded[split:][:]
X_val=X_val_padded[split:][:]
y_test=y_val_padded[:split][:]
x_test=X_val_padded[:split][:]


In [50]:
def run_RNN():
    model=RNN(VOCAB_SIZE, LABELS_SIZE)
    print(model)
    train(model, LSTM_PATH, X_train_padded, y_train_padded)
    
def run_CNN():
    model=CNN(VOCAB_SIZE, LABELS_SIZE)
    print(model)
    train(model, CNN_PATH, X_train_padded, y_train_padded)
    
def run_CNN_eslam():
    model=RNN_CNN(VOCAB_SIZE, LABELS_SIZE)
    print(model)
    train(model,CNN_PATH,CNN_val_PATH, X_train_padded, y_train_padded,X_val,y_val)
    
def run_CRF():
    model=LSTM_CRF(VOCAB_SIZE, LABELS_SIZE)
    print(model)
    train(model, CNN_PATH, X_train_padded, y_train_padded)
def run_CRF_eslam():
    model=RNN_CRF(VOCAB_SIZE, LABELS_SIZE)
    print(model)
    train(model,CRF_PATH,CRF_Val_PATH, X_train_padded, y_train_padded,X_val,y_val)

In [53]:
# run_RNN()
# run_CNN()
# run_CRF()
# run_CRF_eslam()
run_CNN_eslam()

RNN_CNN(
  (embedding): Embedding(37, 200)
  (conv1d): Conv1d(200, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (lstm): LSTM(64, 512, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=1024, out_features=15, bias=True)
)


  0%|          | 0/179 [00:00<?, ?it/s]

100%|██████████| 179/179 [01:09<00:00,  2.56it/s]


Epochs: 1 | Train Loss: 0.0015377842305640016             | Train Accuracy: 0.872101344549541

Saved the best model with accuracy: 0.872101344549541 to ./models/cnn.pth



100%|██████████| 5/5 [00:00<00:00,  6.98it/s]


...

Epochs: 9 | Validation Loss: 0.12437915951013565 | Validation Accuracy: 0.9541864139020537
Saved the best model with validation accuracy: 0.9541864139020537 to ./models/cnn_val.pth


100%|██████████| 179/179 [01:05<00:00,  2.74it/s]


Epochs: 10 | Train Loss: 0.00034676016834323025             | Train Accuracy: 0.968540132162542

Saved the best model with accuracy: 0.968540132162542 to ./models/cnn.pth



100%|██████████| 5/5 [00:00<00:00,  6.23it/s]

Epochs: 10 | Validation Loss: 0.12441576272249222 | Validation Accuracy: 0.9534755134281201
